# Statystyka w Analizie Danych

## Laboratorium 5 - algorytm normalizacja, selekcja cech.


### Opis
Celem laboratorium jest przeprowadzenie normalizacji i selekcji cech.


### Zbiór danych

Zbiór danych znajduje się w katalogu `dataset/*`. Jest to zmodyfikowany zbiór danych znajdujący się pod adresem: <https://archive.ics.uci.edu/ml/datasets/leaf>.

### Przesyłanie zadań

Wszystkie pliki należy spakować archiwizatorem **zip** i przesłać za pośrednictwem platformy WIKAMP. Poniżej oczekiwana zawartość archiwum:

```
+-- 📂 [IMIE.NAZWISKO].zip
    +-- 📜 Lab05.ipynb
    +-- 📂 dataset
        +-- 📜 dataset.npz
        +-- 📜 ReadMe.pdf
```

**Pamiętaj, wyniki powinny być czytelnie opisane oraz zaprezentowane graficznie (jeżeli jest taka możliwość).**

Przykład (na podstawie tablicy pomyłek):

**Źle** (nie wiadomo co jest poniżej zaprezentowane, kolumny ani wiersze nie są podpisane, nie wiadomo które z nich prezentują predykcje, a które właściwe etykiety):
```
array([[2, 0, 0],
       [0, 0, 1],
       [1, 0, 2]])
```

### Zadanie

Należy wykonać następujące czynności w celu realizacji niniejszego zadania:

#### Normalizacja
* Wczytaj dane.
* Znormalizuj dane.
* Przeprowadź eksperyment z zastosowaniem algorytmu kNN lub NM dla danych znormalizowanych oraz bez normalizacji.
    * W eksperymencie wybierz 5 klas oraz 10 cech.
* Przedstaw porównanie wyników klasyfikacji na danych znormalizowanych i bez normalizacji.
* Napisz wnioski.

**UWAGA: Wykorzystaj gotową implementację kNN [KNeighborsClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html)**

In [26]:
# <<< TUTAJ WYKONAJ Z
import numpy as np
from sklearn.neighbors import KNeighborsClassifier

matplotlib_axes_logger.setLevel('ERROR')


with open('dataset.npz', 'rb') as f:
    data = np.load(f)
    train, test = data['train'], data['test']

In [27]:
def normalize(arr):
    return ((arr - arr.min())/(arr.max()-arr.min()))

In [ ]:
def chooseClass(classes):
    for i in range(len(classes)):
        if i == 0:
            new_train = np.array(train[train[:,0] == classes[i]])
            new_test = np.array(test[test[:,0] == classes[i]])
        else:
            new_train = np.vstack((new_train,train[train[:,0] == classes[i]]))
            new_test = np.vstack((new_test,test[test[:,0] == classes[i]]))
    return new_train, new_test

In [28]:
origTrain = np.copy(train)
origTest = np.copy(test)

for i in range(2,16):
    toNormalize = train[:,i]
    train[:,i] = normalize(toNormalize)
    
for i in range(2,16):
    toNormalize = test[:,i]
    test[:,i] = normalize(toNormalize)

In [29]:
xTrain = train[:,2:]
xTest = test[:,2:]
yTrain = train[:,0]
yTest = test[:,0]

array([3.60000000e+01, 1.00000000e+00, 3.25831191e-01, 2.42662236e-02,
       7.03507684e-01, 4.54961599e-01, 4.95453968e-01, 1.86266829e-01,
       3.85108850e-01, 1.53730769e-01, 3.05450423e-01, 5.12751050e-01,
       3.32917414e-01, 3.79754707e-01, 9.76735516e-02, 4.22128651e-01])

#### Selekcja cech
Na tym samym podzbiorze danych (co w poprzednim zadaniu).
* Przeprowadź selekcję cech (wybierz {2, 5} cech) za pomocą metod poznanych na wykładzie (np. z zastosowaniem współczynnika Fishera) lub istniejących implementacji z biblioteki [scikit-learn](https://scikit-learn.org/stable/modules/feature_selection.html) (np. [SelectKBest](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectKBest.html#sklearn.feature_selection.SelectKBest)).
* Przeprowadź klasyfikację na wybranych cechach.
* Porównaj wyniki klasyfikacji dla różnej liczby cech:
    * 10 cech bez normalizacji (wyniki z poprzedniego zadania),
    * 10 cech z normalizacją (wyniki z poprzedniego zadania),
    * 5 wybranych cech z bez normalizacji,
    * 5 wybranych cech z normalizacją,
    * 2 wybranych cech z bez normalizacji,
    * 2 wybranych cech z normalizacją.
* Opisz wyniki i napisz wnioski.


In [ ]:
# <<< TUTAJ WYKONAJ ZADANIE >>>